### Automatically extraction of attributes, types and write the results to latex 

In [1]:
import os
import concurrent.futures
from osgeo import ogr

In [2]:
# Set global variables
root_dir = '../data_sets/' # Root directory
dirs = ['dataset I/','dataset II/','dataset III/','dataset IV/'] # List of directories to be searched
# dirs = ['dataset IV/'] # List of directories to be searched
out_tex_file = 'report/attributes.tex'

field_names_in_file = {} # List to hold the field names
data = {} # Dictionary to hold the data

In [3]:
# Gather all shapefile paths
# shapefiles = []
# for dir in dirs:
#     # Traverse directory
#     for dir_name, subdir_list, file_list in os.walk(root_dir + dir):
#         for fname in file_list:
#             # Check if the file is a shapefile
#             if fname.endswith('.shp'):
#                 shapefiles.append(os.path.join(dir_name, fname))
                

In [4]:
comp_file_dict = {
    'Pipe':['dataset I/EST_OUEST/canalisations_EO',
            'dataset I/RESEAU_MAERA/canalisations',
            'dataset I/RESEAU/canalisations',
            'dataset II/A_TRONCON',
            'dataset III/M3M/A_TRONCON',
            'dataset IV/ass_rassa',
            'dataset IV/ReseauMarcheTopo',
            ],
    'Service Line':['dataset I/EST_OUEST/canalisations_de_branchement_EO', 
                    'dataset I/RESEAU/canalisations_de_branchement',
                    'dataset II/A_BRANCHEMENT_LINE', 
                    'dataset III/M3M/A_BRANCHEMENT',
                    'dataset IV/ass_brat'
                    ],
    'Connection Point':['dataset I/EST_OUEST/branchements_EO', 
                        'dataset I/RESEAU/branchements',
                        'dataset II/A_BRANCHEMENT_POINT',
                        'dataset IV/ass_bran'
                        ],
    'Manhole':['dataset I/EST_OUEST/regards_EO', 
               'dataset I/RESEAU_MAERA/regards_contrat_maera',
               'dataset I/RESEAU/regards',
               'dataset II/A_REGARD', 
               'dataset III/M3M/A_REGARD', 
               'dataset IV/REG'
               ],
    'Fitting':['dataset I/EST_OUEST/raccords_EO', 
               'dataset I/RESEAU_MAERA/raccords_contrat_maera',
               'dataset I/RESEAU/raccords',
               'dataset IV/RAC'
               ],
    'Pump Stattion':['dataset I/EST_OUEST/pompages_EO', 
                     'dataset I/RESEAU_MAERA/pompages_contrat_maera', 
                     'dataset I/RESEAU/pompages', 
                     'dataset IV/PMP'
                     ],
    'Sewage Treatment':['dataset I/EST_OUEST/station_de_traitement_EO', 
                        'dataset I/RESEAU_MAERA/station_de_traitement_contrat_maera', 
                        'dataset I/RESEAU/station_de_traitement',
                        'dataset IV/TRA'
                        ],
    'Structure':['dataset I/EST_OUEST/ouvrages_EO', 
                 'dataset I/RESEAU_MAERA/ouvrages_contrat_maera', 
                 'dataset I/RESEAU/ouvrages', 
                 'dataset II/A_OUVRAGE', 
                 'dataset III/M3M/A_OUVRAGE', 
                 'dataset IV/OUV'
                 ],
    'Accessories': ['dataset II/A_ACCESSOIRE', 
                   'dataset III/M3M/A_ACCESSOIRE'
                   ],
    'Others':['dataset IV/CommuneExploitant',
              'dataset IV/recolementEU'
              ]
    } # Dictionary to hold the file names

In [5]:
# Gather all shapefile paths
shapefiles = []
for comp, files in comp_file_dict.items():
    for f in files:
        shapefiles.append(root_dir + f +'.shp')

In [6]:
for shapefile in shapefiles:
    # Open the shapefile
    ds = ogr.Open(shapefile)
    
    if ds is None:
        print('Could not open file: ' + shapefile)
        continue
    
    lyr = ds.GetLayer()
    
    # Get the field names
    field_names_in_file[shapefile] = [field.name for field in lyr.schema]
        
    # Close the shapefile
    ds = None

In [7]:
# for sh in shapefiles:
#     print(sh)

In [8]:
# Load the shapefile data in root directory and subdirectories

# The function to be run in parallel
def process_shapefile(file_path):
    print('Processing %s' % file_path)
    # Get the field names
    # Open the shapefile
    ds = ogr.Open(file_path)
    
    if ds is None:
        print('Could not open file: ' + file_path)
        return None, None
    
    lyr = ds.GetLayer()
    fld_names = [field.name for field in lyr.schema]

    flds_data = {}
    # Get the data
    lyr.ResetReading()
    for feat in lyr: # For each feature
        for fld in fld_names:
            if fld not in flds_data:
                flds_data[fld] = []
            flds_data[fld].append(feat.GetField(fld))

    return fld_names, flds_data

# Process all shapefiles in parallel
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_file = {executor.submit(process_shapefile, shp): shp for shp in shapefiles}
    for future in concurrent.futures.as_completed(future_to_file):
        file = future_to_file[future]
        # fname = os.path.basename(file)
        fname = file.removeprefix(root_dir)
        try:
            fld_names, flds_data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (file, exc))
        else:
            field_names_in_file[fname] = fld_names
            data[fname] = flds_data


Processing ../data_sets/dataset I/EST_OUEST/canalisations_EO.shp
Processing ../data_sets/dataset I/RESEAU_MAERA/canalisations.shp
Processing ../data_sets/dataset I/RESEAU/canalisations.shp
Processing ../data_sets/dataset II/A_TRONCON.shp
Processing ../data_sets/dataset III/M3M/A_TRONCON.shp
Processing ../data_sets/dataset IV/ass_rassa.shp
Processing ../data_sets/dataset IV/ReseauMarcheTopo.shp
Processing ../data_sets/dataset I/EST_OUEST/canalisations_de_branchement_EO.shp
Processing ../data_sets/dataset I/RESEAU/canalisations_de_branchement.shp
Processing ../data_sets/dataset II/A_BRANCHEMENT_LINE.shp
Processing ../data_sets/dataset III/M3M/A_BRANCHEMENT.shp
Processing ../data_sets/dataset IV/ass_brat.shp
Processing ../data_sets/dataset I/EST_OUEST/branchements_EO.shp
Processing ../data_sets/dataset I/RESEAU/branchements.shp
Processing ../data_sets/dataset II/A_BRANCHEMENT_POINT.shp
Processing ../data_sets/dataset IV/ass_bran.shp
Processing ../data_sets/dataset I/EST_OUEST/regards_EO.s

In [9]:
# Data types utils

import datetime
from dateutil.parser import parse
import dateparser
import numpy as np
from collections import Counter

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
def is_date(s):
    strict_settings = {'STRICT_PARSING': True}
    return bool(dateparser.parse(s, settings=strict_settings))
    
def is_time(s):
    try:
        datetime.datetime.strptime(s, '%H:%M:%S')
        return True
    except ValueError:
        return False
    
def is_datetime(s):
    try:
        datetime.datetime.strptime(s, '%d/%m/%Y %H:%M:%S')
        return True
    except ValueError:
        return False
    
def is_bool(s):
    if s.lower() == 'true' or s.lower() == 'false':
        return True
    else:
        return False
    
def is_null(s):
    if s.lower() == '<null>' or s == None or s.upper() == 'INCONNU':
        return True
    else:
        return False


STRING = 'String'
NUMBER = 'Number'
NULL = 'Null'
DATE = 'Date'
TIME = 'Time'
DATETIME = 'Datetime'
BOOL = 'Bool'
CATEGORICAL = 'Categorical'
UNKNOWN = 'Unknown'

def get_type_of_value(value):
    if is_number(value):
        return NUMBER
    elif is_date(value):
        return DATE
    # elif is_time(value):
    #     return TIME
    # elif is_datetime(value):
    #     return DATETIME
    # elif is_bool(value):
    #     return BOOL
    # elif is_null(value):
    #     return NULL
    else:
        return STRING    

In [10]:
# Get the type from an attribute values list
def get_type(list, threshold=0.9, caterogical_threshold=0.2, categorical_max_values=100, null_values=[None]):
    """Get the type from an attribute values list

    Args:
        list (Any): list of values to be analyzed
        threshold (float, optional): if the number of occurences of a type is greater than the threshold, then it's the type of the attribute. Defaults to 0.9.
        caterogical_threshold (float, optional): number of unique values by the number of elements is less than the threshold, it's an Categorical attribute. Defaults to 0.2.
        categorical_max_values (int, optional): the maximum number that a categorical attribute can have. Defaults to 100.
        null_values (list, optional): list of null values. Defaults to [None].

    Returns:
        [list]: [type, max_value, min_value, values_list]
    """
    
    data_ = np.array(list)
    data_occurence = Counter(data_)
    
    # remove null values
    for key in null_values:
        if key in data_occurence:
            del data_occurence[key]
            
    # nbr of elements
    nbr_of_elemenst = data_occurence.total()
    
    if(nbr_of_elemenst == 0):
        return [UNKNOWN, None, None, None, "0/"+str(len(list))]
    
    # return vars
    type = UNKNOWN
    max_val = None
    min_val = None
    values_list = None
    
    types_dict = {STRING:0, NUMBER:0, NULL:0, DATE:0}
    
    values_count = len(data_occurence)
    
    # this takes a lot of time
    for val,occ in data_occurence.items():
        val_type = get_type_of_value(val)
        types_dict[val_type] += occ
        
        # if the number of occurences of STRING is greater than the (1 - threshold), then it's the type could be STRING or CATEGORICAL
        if val_type == STRING and types_dict[STRING] / nbr_of_elemenst > (1 - threshold):
            type = STRING
            break
        
        # if the number of occurences of a type is greater than the threshold, then it's the type
        if types_dict[val_type] / nbr_of_elemenst > threshold :
            type = val_type
            if val_type == NUMBER:
                numbers = [k for (k,v) in data_occurence.items() if is_number(k)]
                max_val = max(numbers)
                min_val = min(numbers)
            break
    
    
    if nbr_of_elemenst > 0 and (values_count / nbr_of_elemenst) < caterogical_threshold and values_count <= categorical_max_values:
        if type != None:
            type = CATEGORICAL + '-' + type
        else:
            type = CATEGORICAL
            
        values_list = []
        for val,occ in data_occurence.items():
            values_list.append(val)
        
    return [type, max_val, min_val, values_list, str(nbr_of_elemenst)+"/"+str(len(list))]

In [11]:
# Get the types of all fields in parallel

from concurrent.futures import ProcessPoolExecutor, as_completed

def process_field(fname, fld):
    attr_type, max_val, min_val, values_list, not_null_values = get_type(data[fname][fld], threshold=0.9, caterogical_threshold=0.2, categorical_max_values=10, null_values=['<Nul>', None, 'INCONNU'])
    type = {'type': attr_type, 'max': max_val, 'min': min_val, 'values': values_list, 'not_null_values':not_null_values}
        
    return [fname, fld, type]

fld_types = {}
with ProcessPoolExecutor() as executor:
    futures = []
    for fname in data:
        for fld in data[fname]:
            # print('Processing field {} of file {}'.format(fld, fname))
            futures.append(executor.submit(process_field, fname, fld))
            
    for type in as_completed(futures):
        fname, fld, result = type.result()
        if fname not in fld_types:
            fld_types[fname] = {}
        fld_types[fname][fld] = result


In [12]:
fld_types_by_file = {}
for fname in fld_types:
    fld_types_by_file[fname] = {}
    for fld in fld_types[fname]:
        fld_types_by_file[fname][fld] = fld_types[fname][fld]

In [13]:
for file in fld_types_by_file:
    print(file)
    for fld in fld_types_by_file[file]:
        print('\t{}: {}'.format(fld, fld_types_by_file[file][fld]))

dataset I/RESEAU_MAERA/canalisations.shp
	BANQUETTE: {'type': 'Categorical-Number', 'max': '1', 'min': '1', 'values': ['1'], 'not_null_values': '186/236'}
	OBJECTID: {'type': 'Number', 'max': 74665, 'min': 0, 'values': None, 'not_null_values': '236/236'}
	F_ID_NODE: {'type': 'Number', 'max': 102770, 'min': 63745, 'values': None, 'not_null_values': '236/236'}
	FFIL: {'type': 'Number', 'max': 999.0, 'min': -2.87, 'values': None, 'not_null_values': '236/236'}
	FCHAUSSEE: {'type': 'Number', 'max': 999.0, 'min': 0.0, 'values': None, 'not_null_values': '236/236'}
	DIMENSIONS: {'type': 'Number', 'max': '999', 'min': '100', 'values': None, 'not_null_values': '236/236'}
	FRADIER: {'type': 'Number', 'max': 999.0, 'min': -2.87, 'values': None, 'not_null_values': '236/236'}
	ID_ARC: {'type': 'Number', 'max': 102914, 'min': 63536, 'values': None, 'not_null_values': '236/236'}
	IMPLANT: {'type': 'Unknown', 'max': None, 'min': None, 'values': None, 'not_null_values': '0/236'}
	LONGUEUR: {'type': 'Num

In [14]:
# Nbr of null attributes by file
for comp,files in comp_file_dict.items():
    print(comp)
    for file in files:
        null_count = 0
        shapefile = file + '.shp'
        for fld in fld_types_by_file[shapefile]:
            if fld_types_by_file[shapefile][fld]['type'] == 'Unknown':
                null_count += 1
        print('\t{}: {} by {}'.format(file.removesuffix('/' + file.split('/')[-1]), null_count, len(fld_types_by_file[shapefile])))

Pipe
	dataset I/EST_OUEST: 6 by 48
	dataset I/RESEAU_MAERA: 5 by 48
	dataset I/RESEAU: 6 by 48
	dataset II: 0 by 15
	dataset III/M3M: 0 by 8
	dataset IV: 1 by 54
	dataset IV: 5 by 55
Service Line
	dataset I/EST_OUEST: 3 by 23
	dataset I/RESEAU: 3 by 23
	dataset II: 0 by 9
	dataset III/M3M: 0 by 5
	dataset IV: 1 by 26
Connection Point
	dataset I/EST_OUEST: 4 by 16
	dataset I/RESEAU: 5 by 16
	dataset II: 0 by 9
	dataset IV: 3 by 21
Manhole
	dataset I/EST_OUEST: 6 by 28
	dataset I/RESEAU_MAERA: 10 by 28
	dataset I/RESEAU: 6 by 28
	dataset II: 0 by 14
	dataset III/M3M: 0 by 7
	dataset IV: 4 by 39
Fitting
	dataset I/EST_OUEST: 7 by 18
	dataset I/RESEAU_MAERA: 6 by 18
	dataset I/RESEAU: 6 by 18
	dataset IV: 6 by 21
Pump Stattion
	dataset I/EST_OUEST: 4 by 58
	dataset I/RESEAU_MAERA: 25 by 58
	dataset I/RESEAU: 8 by 58
	dataset IV: 5 by 66
Sewage Treatment
	dataset I/EST_OUEST: 8 by 17
	dataset I/RESEAU_MAERA: 10 by 17
	dataset I/RESEAU: 10 by 17
	dataset IV: 8 by 31
Structure
	dataset I/EST_

### Write data to Latex file

In [15]:
similar_attributes = {} # {(attr, type): file}

In [21]:
# write_data_2_latex_items
def write_data_2_latex_items(file_out, file_name, attributes):
    if attributes is None:
        return
    
    subsection = file_name.split('/')[-1] + ' from ' + '\\textit{' + file_name.removesuffix(file_name.split('/')[-1])[:-1] + '}'
    
    file_out.write(('\\subsection{' + subsection + '}\n').replace('_', '\_'))
    file_out.write('\\begin{description}\n')
    
    attr_nullVals = []
    
    for attr in attributes:
        # Attribute Name
        line = '\\item[' + '\\textbf{' + attr +  ':}] ' 
        
        # Type
        line += '\\textit{Type}: ' + str(attributes[attr]['type'])
        
        # Min, Max
        if attributes[attr]['min'] is not None and attributes[attr]['max'] is not None:
            line += ', \\textit{Min-Max}: [' + str(attributes[attr]['min']) + ',' + str(attributes[attr]['max']) + ']'
        
        # Nbr null values
        line += ', \\textit{Not null values}: ' + str(attributes[attr]['not_null_values'])
        
        # Values:{ vals}
        if attributes[attr]['values'] is not None:
            line += ', \\textit{Values}: \{'
            line += str(attributes[attr]['values'][0])
            for val in attributes[attr]['values'][1:]:
                line += ', ' + str(val)
            line += '\}'
            
        
        # Write the line to file
        file_out.write(line.replace('_', '\_') + '\n')
        attr_nullVals.append([attr, attributes[attr]['not_null_values']])
        

    file_out.write('\\end{description}\n')
    
    # create a table with the null values  | Attribute | Null values | Attribute | Null values |
    file_out.write('The following table shows the number of null values for each attribute: \n')
    file_out.write('\\begin{table}[H]\n')
    file_out.write('\\centering\n')
    file_out.write('\\begin{tabular}{|c|c|c|c|}\n')
    file_out.write('\\hline\n')
    file_out.write('\\rowcolor{gray!25}')
    file_out.write('\\textbf{Attribute} & \\textbf{Filled values} & \\textbf{Attribute} & \\textbf{Filled values} \\\\ \\hline\n')
    
    green = "\\cellcolor{green!50}"
    red = "\\cellcolor{red!50}"
    
    for i in range(0, len(attr_nullVals), 2):
        attr1 = attr_nullVals[i][0]
        vals1_str = attr_nullVals[i][1].split('/')
        vals1 = [int(vals1_str[0]), int(vals1_str[1])]
        perc1 = "N/A"
        cell1_color = ""

        if vals1[1] != 0:
            if vals1[0]/vals1[1] > 0.9:
                cell1_color = green
            elif vals1[0]/vals1[1] < 0.3:
                cell1_color = red
                
            if vals1[0]/vals1[1] > 0.9999 and vals1[0]/vals1[1] < 1:
                perc1 = "~100\%"
            else:
                perc1 = "{:.2f}".format(vals1[0]/vals1[1]*100) + "\%"
        
        
        if i+1 == len(attr_nullVals):
            file_out.write(cell1_color + attr1.replace('_', '\_') + ' & ' + perc1 + ' & & \\\\ \\hline\n')
            break
        
        attr2 = attr_nullVals[i+1][0]
        vals2_str = attr_nullVals[i+1][1].split('/')
        vals2 = [int(vals2_str[0]), int(vals2_str[1])]
        perc2 = "N/A"
        cell2_color = ""
        
        if vals2[1] != 0:
            if vals2[0]/vals2[1] > 0.9:
                cell2_color = green
            elif vals2[0]/vals2[1] < 0.3:
                cell2_color = red
                
            if vals2[0]/vals2[1] > 0.9999 and vals2[0]/vals2[1] < 1:
                perc2 = "~100\%"
            else:
                perc2 = "{:.2f}".format(vals2[0]/vals2[1]*100) + "\%"
        
        file_out.write(cell1_color + attr1.replace('_', '\_') + ' & ' + perc1 + ' & ' + cell2_color + attr2.replace('_', '\_') + ' & ' + perc2 + ' \\\\ \\hline\n')
        
    file_out.write('\\end{tabular}\n')
    file_out.write('\\caption{Null values for each attribute}\n')
    file_out.write('\\label{tab:null_values}\n')
    file_out.write('\\end{table}\n')
    
    



In [22]:
# Write the data to latex file
with open(out_tex_file, 'w') as file_out:
    for comp,files in comp_file_dict.items():
        if len(files) > 0:
            file_out.write(('\\section{' + comp + '}\n').replace('_', '\_'))
        for file in files:
                if (file + '.shp') in fld_types_by_file:
                    write_data_2_latex_items(file_out, file, fld_types_by_file[file + '.shp'])
                else:
                    print('File {} not found'.format(file))